In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000,
                                    bootstrap=True,
                                    max_features='sqrt')

try:
    f = open('final_model.sav')
    classifier = pickle.load(open('final_model.sav', 'rb'))

except:
    import pandas as pd
    import numpy as np
    import re

    # from wordcloud import WordCloud, STOPWORDS
    # from collections import Counter
    #import scispacy
    #import spacy

    # importing
    df = pd.read_csv('df_diseases.csv')

    # dropping col's
    df.drop([df.columns[0], df.columns[2]], axis=1, inplace=True)

    # Filling NaN values with empty string
    df.fillna('', inplace=True)

    # some pre-processing
    for i in range(len(df)):

        df.loc[i, 'symptoms'] = re.sub(
            '[^A-Za-z0-9., ]+', '', df.loc[i, 'symptoms'])
        df.loc[i, 'causes'] = re.sub(
            '[^A-Za-z0-9., ]+', '', df.loc[i, 'causes'])
        df.loc[i, 'risk_factor'] = re.sub(
            '[^A-Za-z0-9., ]+', '', df.loc[i, 'risk_factor'])

        df.loc[i, 'overview'] = re.sub(
            '[^A-Za-z0-9., ]+', '', df.loc[i, 'overview'])
        df.loc[i, 'treatment'] = re.sub(
            '[^A-Za-z0-9., ]+', '', df.loc[i, 'treatment'])
        df.loc[i, 'medication'] = re.sub(
            '[^A-Za-z0-9., ]+', '', df.loc[i, 'medication'])

        df.loc[i, 'home_remedies'] = re.sub(
            '[^A-Za-z0-9., ]+', '', df.loc[i, 'home_remedies'])

    # some more preprocessing
    df['name'] = df['name'].str.lower()
    df['symptoms'] = df['symptoms'].str.lower()
    df['causes'] = df['causes'].str.lower()

    df['risk_factor'] = df['risk_factor'].str.lower()
    df['overview'] = df['overview'].str.lower()
    df['treatment'] = df['treatment'].str.lower()

    df['medication'] = df['medication'].str.lower()
    df['home_remedies'] = df['home_remedies'].str.lower()

    # adding a new column named 'class' which will contain the category of the disease
    df['class'] = -1

    def create_classes(row):
        if((row['medication'] == '') and (row['home_remedies'] == '')):
            return 1
        elif((row['medication'] != '') and (row['home_remedies'] == '')):
            return 2
        elif((row['medication'] != '') and (row['home_remedies'] != '')):
            return 3
        elif((row['medication'] == '') and (row['home_remedies'] != '')):
            return 4

    df['class'] = df.apply(create_classes, axis=1)

    test_df = df.copy()

    test_df['input'] = ''

    for i in range(len(test_df)):
        test_df.loc[i, 'input'] = test_df.loc[i, 'symptoms']+test_df.loc[i,
                                                                         'causes']+test_df.loc[i, 'risk_factor']+test_df.loc[i, 'overview']

    # Training model on entire dataset
    X = test_df['input'].values

    Y = test_df['class'].values

    from sklearn.feature_extraction.text import CountVectorizer

    count_vect = CountVectorizer(decode_error="replace").fit(X)

    file_name_for_vector = 'feature.pkl'

    pickle.dump(count_vect.vocabulary_, open(file_name_for_vector, 'wb'))

    X_counts = count_vect.transform(X)

    # TF-IDF
    from sklearn.feature_extraction.text import TfidfTransformer
    tfidf_transformer = TfidfTransformer().fit(X_counts)

    X_tfidf = tfidf_transformer.transform(X_counts)

    classifier = RandomForestClassifier(n_estimators=1000,
                                        bootstrap=True,
                                        max_features='sqrt').fit(X_tfidf, Y)

    filename = 'final_model.sav'

    pickle.dump(classifier, open(filename, 'wb'))

In [2]:
def make_prediction(input1):
    input_arr = np.array([input1])
    ser = pd.Series(input_arr)
    new_input = ser.values

    transformer = TfidfTransformer()

    file_name_for_vector = 'feature.pkl'

    loaded_vec = CountVectorizer(
        decode_error="replace", vocabulary=pickle.load(open(file_name_for_vector, "rb")))

    input_tfidf = transformer.fit_transform(loaded_vec.transform(new_input))

    return classifier.predict(input_tfidf)


x = 'fever cough cold'

print(make_prediction(x))


[1]
